In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from load_data import load_data, split_data

data = load_data("1H")
all_columns = data.keys()
all_columns

Index(['MWh', 'solar_fore_de [MW]', 'solar_fore_it [MW]', 'wind_fore_de [MW]',
       'wind_fore_it [MW]', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH',
       'DE_CH', 'FR_CH', 'IT_CH'],
      dtype='object')

In [2]:
columns_to_drop = [
    "CH_AT",
    "CH_DE",
    "CH_FR",
    "CH_IT",
    "AT_CH",
    "DE_CH",
    "FR_CH",
    "IT_CH",
]

In [3]:
data_filtered = data.drop(columns=columns_to_drop)
data_filtered

,MWh,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW]
2019-01-01 00:00:00+00:00,129.716036,0.0,0.0,23052.3310,4596.5916
2019-01-01 01:00:00+00:00,133.398074,0.0,0.0,24969.9701,4478.5564
2019-01-01 02:00:00+00:00,135.133852,0.0,0.0,27082.9626,4323.3712
2019-01-01 03:00:00+00:00,131.699424,0.0,0.0,26890.9717,4231.8283
2019-01-01 04:00:00+00:00,147.391128,0.0,0.0,27740.1555,4266.3082
...,...,...,...,...,...
2021-12-31 18:00:00+00:00,171.707318,0.0,0.0,36997.7200,1108.4000
2021-12-31 19:00:00+00:00,159.462903,0.0,0.0,35666.9300,1077.9700
2021-12-31 20:00:00+00:00,155.109520,0.0,0.0,34383.8800,1048.2800
2021-12-31 21:00:00+00:00,171.370277,0.0,0.0,33075.2500,1078.7800


In [4]:
sep = len(data_filtered) // 3 * 2

train_df = data_filtered[: sep]
test_df = data_filtered[sep :]

X_train, metadata_train, y_train = split_data(train_df, 7 * 24, 24, 1, "MWh")
X_test, metadata_test, y_test = split_data(test_df, 7 * 24, 24, 1, "MWh")

X_train.shape, metadata_train.shape, y_train.shape, X_test.shape

((17343, 168), (17343, 5, 24), (17343, 24), (8578, 168))

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Check CUDA and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# Create DataLoader for mini-batch gradient descent
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define DNN model with dropout layers
class DNN(nn.Module):
    def __init__(self, input_dim):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.2)
        self.layer3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout(x)
        x = torch.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.layer3(x)
        return x

# Initialize model, loss, and optimizer
input_dim = X_train.shape[1]
model = DNN(input_dim).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping initialization
min_val_loss = float('inf')
patience = 10
counter = 0

# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        y_pred = model(batch_x)
        loss = criterion(y_pred.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Early stopping based on validation loss
    model.eval()
    with torch.no_grad():
        val_pred = model(X_test_tensor)
        val_loss = criterion(val_pred.squeeze(), y_test_tensor)
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            counter = 0  # reset counter
        else:
            counter += 1  # increment counter
            if counter >= patience:
                print("Early stopping triggered")
                break

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor).cpu().numpy()
    mae = mean_absolute_error(y_test, y_test_pred)
    print(f"Mean Absolute Error: {mae}")



/opt/tljh/user/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([32, 24])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (32) must match the size of tensor b (24) at non-singleton dimension 1

In [ ]:
train_mae_list[0].item()